In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv


In [23]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [24]:
# загружаем данные
app_train = pd.read_csv('/kaggle/input/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('/kaggle/input/home-credit-default-risk/application_test.csv')
sample_submission = pd.read_csv('/kaggle/input/home-credit-default-risk/sample_submission.csv')

## Шаг 1. Начнём. Притащим сюда модели из других моих ноутбуков

### 1.1 Логистическая регрессия

**Препроцессинг:**

In [25]:
good_features = ['AMT_GOODS_PRICE', 'AMT_ANNUITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'DAYS_EMPLOYED', 'DAYS_BIRTH']

# заполняем пропуски
features_list = []

for feature in good_features:
    if (app_train[feature].isnull().sum() != 0):
        features_list.append(feature)
for feature in features_list:
    good_features.remove(feature)

In [26]:
#features_list = ['AMT_GOODS_PRICE', 'AMT_ANNUITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']

for feature in features_list:
    mean = app_train[feature].mean()
    app_train[feature + '_filled'] = app_train[feature].fillna(mean)
    
    mean = app_test[feature].mean()
    app_test[feature + '_filled'] = app_test[feature].fillna(mean)
    feature = feature + '_filled'
    good_features.append(feature)

# логарифмируем признаки, чтобы привести к нормальному распределению
app_train['AMT_INCOME_TOTAL_log'] = np.log1p(app_train['AMT_INCOME_TOTAL'])
app_train = app_train.loc[app_train['AMT_INCOME_TOTAL_log'] < 14] #убираю выбросы
app_test['AMT_INCOME_TOTAL_log'] = np.log1p(app_test['AMT_INCOME_TOTAL'])

app_train['AMT_CREDIT_log'] = np.log1p(app_train.loc[:,['AMT_CREDIT']])
app_test['AMT_CREDIT_log'] = np.log1p(app_test['AMT_CREDIT'])

app_train['AMT_GOODS_PRICE_filled_log'] = np.log1p(app_train.loc[:,['AMT_GOODS_PRICE_filled']])
app_test['AMT_GOODS_PRICE_filled_log'] = np.log1p(app_test['AMT_GOODS_PRICE_filled'])

app_train['AMT_ANNUITY_filled_log'] = np.log1p(app_train.loc[:,['AMT_ANNUITY_filled']])
app_test['AMT_ANNUITY_filled_log'] = np.log1p(app_test['AMT_ANNUITY_filled'])

for f in ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_GOODS_PRICE_filled', 'AMT_ANNUITY_filled']:
    good_features.remove(f)
    good_features.append(f + '_log')

# Добавим ещё один признак, производный от первых двух: 
# TIME_TO_PAY = AMT_INCOME_TOTAL_log/AMT_CREDIT_log
app_train['TIME_TO_PAY_log'] = app_train['AMT_INCOME_TOTAL_log']/app_train['AMT_CREDIT_log']
app_test['TIME_TO_PAY_log'] = app_test['AMT_INCOME_TOTAL_log']/app_test['AMT_CREDIT_log']

good_features.append('TIME_TO_PAY_log')

In [27]:
print(good_features)

['DAYS_EMPLOYED', 'DAYS_BIRTH', 'EXT_SOURCE_1_filled', 'EXT_SOURCE_2_filled', 'EXT_SOURCE_3_filled', 'AMT_INCOME_TOTAL_log', 'AMT_CREDIT_log', 'AMT_GOODS_PRICE_filled_log', 'AMT_ANNUITY_filled_log', 'TIME_TO_PAY_log']


In [28]:
# нормируем признаки
scaler = preprocessing.StandardScaler()

to_norm = []
for f in good_features:
    to_norm.append(f)

for feature in to_norm:
    scaler.fit(app_train[[feature]])
    app_train[[feature + '_norm']] = scaler.transform(app_train[[feature]])
    app_test[[feature + '_norm']] = scaler.transform(app_test[[feature]])
    good_features.remove(feature)
    good_features.append(feature + '_norm')
print(good_features)

['DAYS_EMPLOYED_norm', 'DAYS_BIRTH_norm', 'EXT_SOURCE_1_filled_norm', 'EXT_SOURCE_2_filled_norm', 'EXT_SOURCE_3_filled_norm', 'AMT_INCOME_TOTAL_log_norm', 'AMT_CREDIT_log_norm', 'AMT_GOODS_PRICE_filled_log_norm', 'AMT_ANNUITY_filled_log_norm', 'TIME_TO_PAY_log_norm']


In [29]:
# делаем lable encoding для категориальных признаков
for col in ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    le = preprocessing.LabelEncoder()
    le.fit(app_train[col])
    app_train[col] = le.transform(app_train[col])
    app_test[col] = le.transform(app_test[col])

# Encoding с One Hot Encoder
#NAME_INCOME_TYPE + ORGANIZATION_TYPE
one_hot1 = pd.get_dummies(app_train['NAME_INCOME_TYPE'])
one_hot2 = pd.get_dummies(app_train['ORGANIZATION_TYPE'])
new_train = pd.concat([app_train,one_hot1,one_hot2], axis = 1)

one_hot1 = pd.get_dummies(app_test['NAME_INCOME_TYPE'])
one_hot2 = pd.get_dummies(app_test['ORGANIZATION_TYPE'])
new_test = pd.concat([app_test,one_hot1,one_hot2], axis = 1)

new_train, new_test = new_train.align(new_test, join = 'inner', axis = 1)
new_train['TARGET'] = app_train['TARGET']


**Сама модель лог-регрессии**

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
features3 = ['EXT_SOURCE_2_filled_norm',
             'AMT_INCOME_TOTAL_log_norm','AMT_CREDIT_log_norm', 'TIME_TO_PAY_log_norm',
             'AMT_GOODS_PRICE_filled_log']
features3 = good_features + list(one_hot1)
#'DAYS_EMPLOYED', 'EXT_SOURCE_3_filled','EXT_SOURCE_1_filled','DAYS_BIRTH''AMT_ANNUITY_filled'

In [32]:
log_model = LogisticRegression(max_iter = 1000)

crosval = cross_val_score(log_model, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("log_model:",np.mean(crosval))

# log_model: 0.6781624709571392 при max_iter = 1000

log_model: 0.7318468894761978


# 1.2 Случайный лес

In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
forest_model_2 = RandomForestClassifier(random_state=2020, n_estimators = 50, max_depth = 4)
crosval = cross_val_score(forest_model_2, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("forest_model_2:",np.mean(crosval))
#forest_model: 0.5964431371465481
#forest_model_2: 0.6769968730270228
#forest_model_2: 0.7088871309721058
#forest_model_2: 0.7155592209105384 при features2 + features
#forest_model_2: 0.7188116787406422
#forest_model_2: 0.7201468633164378

forest_model_2: 0.7201454658968287


## 1.3 Gradient boost

In [36]:
from lightgbm import LGBMClassifier

In [43]:
boost_model_2 = LGBMClassifier(random_state=2020, n_estimators = 25, max_depth = 5, 
                             learning_rate = 0.1)
crosval = cross_val_score(boost_model_2, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("boost_model_2:",np.mean(crosval))
#boost_model: 0.7307067571041472
#boost_model: 0.7445376991869004

boost_model_2: 0.7354469211743099


In [44]:
boost_model_3 = LGBMClassifier(random_state=2020, n_estimators = 50, max_depth = 4, 
                             learning_rate = 0.1)
crosval = cross_val_score(boost_model_3, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("boost_model_3:",np.mean(crosval))

boost_model_3: 0.7406190842761635


# 2. Реализуем stacking

In [ ]:
from sklearn.ensemble import StackingClassifier

In [46]:
#stacking 1
estimators = [('rf1', RandomForestClassifier(random_state=2020, n_estimators = 40, max_depth = 5)),
              ('rf2', RandomForestClassifier(random_state=2035, n_estimators = 50, max_depth = 4)),
              ('gb1', LGBMClassifier(random_state=2020, n_estimators = 25, max_depth = 5, learning_rate = 0.1)),
              ('gb2', LGBMClassifier(random_state=2020, n_estimators = 50, max_depth = 4, learning_rate = 0.1)),
              ('log', LogisticRegression(max_iter = 1000))]

stacking_model = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())

crosval = cross_val_score(stacking_model, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("stacking_model1:",np.mean(crosval))

stacking_model1: 0.7406002378062608


In [45]:
#stacking 2
estimators = [('rf1', RandomForestClassifier(random_state=2020, n_estimators = 40, max_depth = 5)),
              ('rf2', RandomForestClassifier(random_state=2035, n_estimators = 50, max_depth = 4)),
              ('gb1', LGBMClassifier(random_state=2020, n_estimators = 25, max_depth = 5, learning_rate = 0.1)),
              ('log', LogisticRegression(max_iter = 1000))]

stacking_model = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())

crosval = cross_val_score(stacking_model, new_train[features3],  y = new_train['TARGET'], cv = 5, scoring= 'roc_auc')
print("stacking_model2:",np.mean(crosval))

boost_model: 0.7369046809259324


В случае, если убрать из stacking'а лучший алгоритм, результат заметно уменьшается на 0,003. В этот раз стэкинг показал результат лучший, чем лучший из составляющих его алгоритмов.

Результат ухудшился из-за того, что пропал алгоритм чаще других указывал верный результат и давал хорошие данные для обучения финальной модели

P.S. Кажется, что можно было не делать отдельный ноутбук для стекинга, а сделать в том же, что и прошлый. В итоге оставил так, так как в этом ноутбуке неплохо получилось сократить кол-во кода для препроцессинга по сравнению с тем, что был для лог-регрессии